# 합성곱신경망(Convolutional Neural Network) ==> CNN

- 주로 시각적 이미지를 분석하는 데 사용되는데 머신러닝의 한 유형인 딥러닝에서 가장 많이 사용되고 있는 알고리즘.

- 이미지 전체를 작은 단위로 쪼개어 각 부분을 분석하는 것이 핵심
- 이미지를 인식하기 위해 패턴을 찾는 데 유용
- 휴대폰 잠금해제 인식이나 자율 주행 자동차와 같은 분야
- 응용 분야에 따라 CNN을 처음부터 만들 수도 있고, 데이터셋으로 사전 학습된 모델을 사용할 수도 있다.
- CNN은 다른 신경망과 마찬가지로 입력 계층, 출력 계층 및 두 계층 사이의 여러 은닉 계층으로 구성됨.
- 각 계층은 해당 데이터만이 갖는 특징을 학습하기 위해 데이터를 변경하는 계산을 수행
- 가장 자주 사용되는 계층으로는 컨벌루션, 활성화/ReLU, 풀링
    - 컨벌루션: 각 이미지에서 특정 특징을 활성화하는 컨벌루션 필터 집합에 입력 이미지를 통과시킨다.
    - ReLU(Rectified Linear Unit)는 음수 값을 0에 매핑하고 양수 값을 유지하여 더 빠르고 효과적인 학습을 수행
        - 이 때 활성화된 특징만 다음 계층으로 전달되기 때문에 이 과정을 활성화라 부르기도 한다.
    - 풀링은 비선형 다운 샘플링을 수행하고 네트워크에서 학습해야 하는 매개 변수 수를 줄여서 출력을 간소화 함
        - 이 과정을 수백개의 계층에서 반복 수행하면서 여러 특징들을 검출하는 방법

### 완전 연결층
- 단점: 공간 정보 손실
- 1차원 배열의 형태로 데이터를 변환한 후 학습.
    - 이미지 픽셀간의 관계를 고려하지 않는다.
    - 이미지 원래의 특성을 상실
    - 같은 이미지를 좌우 반전시키면 다른 이미지로 인식한다.
    
### 컨볼루션 층
- 장점: 공간 정보 유지
- 이미지 픽셀 사이의 관계를 고려한다.
    - 각 픽셀의 지역적인 특징을 통해 이미지를 판단한다.
    - 같은 이미지를 좌우 반전시키더라도 같은 이미지로 인식한다.
- 완전 연결층에 비해 파라미터 수가 적다.
- 학습 과정에서 이미지 필터를 적용함. (가장자리 검출, 가로선 검출, 세로선 검출 등)
    - CNN을 사용하면서 우리가 직접 적용할 필요는 없다.
    - 학습을 통해 적절히 조절된다.

### 패키지 참조 및 데이터 준비

In [1]:
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.utils import plot_model        # 이미지로.. 뭘 저장하는 거
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(777)

In [2]:
# 학습데이터(train)과 검증데이터(test)를 가져온다.
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
# 가져온 데이터의 차수만 확인
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [4]:
# Fashion-MNIST의 레이블에 해당하는 품목을 리스트로 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
              'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### 훈련/테스트 데이터 준비

In [5]:
# 추출할 랜덤값의 수
sample_size = 9

# 0 - 59999의 범위에서 무작위로 3개의 정수 추출
random_idx = np.random.randint(60000, size = sample_size)

# 각 데이터의 값을 0-1 범위로 만들기 위해서 255(이미지 픽셀 수)로 나눔
# 신경망의 input 데이터로 활용하기 위해 데이터의 차수를 단순화 시킴
x_train = np.reshape(x_train / 255, (-1, 28, 28, 1))
x_test = np.reshape(x_test / 255, (-1, 28, 28, 1))

In [6]:
print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [7]:
# 각 데이터의 Label을 범주형 형태로 변경
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [8]:
# 훈련/테스트 데이터를 0.7/0.3의 비율로 분리
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.3, random_state = 777)

print('Fashion-MNIST ready')

Fashion-MNIST ready


In [9]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(42000, 28, 28, 1) (42000, 10)
(18000, 28, 28, 1) (18000, 10)


### 모델 구성하기
- 풀링 연산(아래 Sequential 코드): 주로 최대 풀링을 사용한다.
    - 평균 풀링
    - 최대 풀링


- 기존처럼 model 객체의 add 함수를 사용해서 층을 구성할 수도 있지만 객체 생성 과정에서 리스트 형태의 파라미터로 층을 구성할 수도 있다.
- Conv2D, MaxPool2D -> 이미지의 특징 하나를 추출하는 계층
- Dense -> 분류 작업 수행

- Conv2D: 이미지를 설정된 조각(filters)만큼 인지하는 과정.
    - filters: 특징 맵의 차원(첫 번째 줄에서 filters = 16이니까 16개 단위로 쪼개서 인식을 한다.)
    - kernal_size: filter의 크기 설정 (튜플(x, y) 단위로 주어야 하는데 단일값 k의 경우 (k, k)로 인식함.) 
    - strides: 스트라이드의 크기를 설정. 기본값: (1, 1)
        - 스트라이드는 입력 데이터에 필터를 적용할 때 이동할 간격, 즉 필터가 이동할 간격.
        - 출력 데이터의 크기를 조절하기 위해 사용한다. 스트라이드(Stride)는 보통 1과 같이 작은 값이 더 잘 작동한다.
    - padding: 합성곱 연산을 수행하기 전, 입력데이터 주변을 특정값으로 채워 늘리는 것.
        - 'same': 출력 형태와 입력 형태가 동일하게 설정된다. 
        - 'valid': 패딩을 사용하지 않는다.(기본값)
        - 주로 출력데이터의 공간적(Spatial) 크기를 조절하기 위해 사용
- MaxPool2D: 인지한 조각(filters) 중에서 가장 큰 값만 뽑아낸다. (Max Pooling)
    - 파라미터를 따로 설정하지 않으면 Conv2D의 파라미터 값을 그대로 가져온다.

In [10]:
model = Sequential([
    # 항상 모델의 첫 번째 층은 입력의 형태를 명시해야 한다.
    # 각 Conv2D층에서 이미지를 파악한다. - 이미지를 조각내는 과정
    Conv2D(filters = 16, kernel_size = 3, strides = (1,1), padding = 'same', activation = 'relu', 
          input_shape = (28, 28, 1)),
    MaxPool2D(pool_size = (2, 2), strides = 2, padding = 'same'),
    Conv2D(filters = 32, kernel_size = 3, strides = (1,1), padding = 'same', activation = 'relu'),
    MaxPool2D(pool_size = (2,2), strides = 2, padding = 'same'),
    Conv2D(filters = 64, kernel_size = 3, strides = (1,1), padding = 'same', activation = 'relu'),
    MaxPool2D(pool_size = (2,2), strides = 2, padding = 'same'),
    Flatten(),   # Dense층에 입력하기 위해 데이터를 펼쳐준다.
    Dense(64, activation = 'relu'),
    # 10개의 출력을 가지는 신경망 -> 분류 문제는 분류하고자 하는 범주 수만큼 최종 출력 수를 지정.
    #                              회귀 문제는 float 형식의 %값이므로 최종 출력은 1개.
    Dense(10, activation = 'softmax')
])

### 모델 학습하기

In [11]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

In [12]:
model.fit(x_train, y_train, epochs = 30, batch_size = 32, validation_data = (x_val, y_val))

Epoch 1/30
1313/1313 [==============================] - 32s 24ms/step - loss: 0.5037 - acc: 0.8158 - val_loss: 0.3509 - val_acc: 0.8727
Epoch 2/30
1313/1313 [==============================] - 31s 23ms/step - loss: 0.3186 - acc: 0.8831 - val_loss: 0.2959 - val_acc: 0.8925
Epoch 3/30
1313/1313 [==============================] - 31s 23ms/step - loss: 0.2729 - acc: 0.8997 - val_loss: 0.2842 - val_acc: 0.8965
Epoch 4/30
1313/1313 [==============================] - 31s 24ms/step - loss: 0.2421 - acc: 0.9105 - val_loss: 0.2554 - val_acc: 0.9082
Epoch 5/30
1313/1313 [==============================] - 33s 25ms/step - loss: 0.2196 - acc: 0.9182 - val_loss: 0.2509 - val_acc: 0.9081
Epoch 6/30
1313/1313 [==============================] - 32s 24ms/step - loss: 0.1990 - acc: 0.9260 - val_loss: 0.2329 - val_acc: 0.9139
Epoch 7/30
1313/1313 [==============================] - 34s 26ms/step - loss: 0.1818 - acc: 0.9324 - val_loss: 0.2440 - val_acc: 0.9127
Epoch 8/30
1313/1313 [==========================

### 모델 구조 확인하기

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0

### plot_model 함수를 이용
- 각주의 해결방법을 써도 통하지 않는다면, 다음 코드의 주석을 풀어 실행시키기

In [25]:
# import os
# os.environ['PATH'] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.42.1/bins'

In [ ]:
plot_model(model, to_file = './model.png', show_shapes = True)